<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Decision Tree   
<font color=black size=4>
    Hamidreza Yaghoubi 
    
<br><br>
<font size=4>
In this homework, we are going to implement the Classification Decision Tree. Keep in mind to complete all of the following questions and write your own codes in the TODO cells.

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 2: Classification Decision Tree (100 points)
We will implement a Classification Decision Tree from scratch in the following problem. Then we will use our model to predict malignant and benign breast cancer. For this purpose, we will use the breast_cancer.csv dataset which you can find more details about it <a href="https://www.kaggle.com/datasets/merishnasuwal/breast-cancer-prediction-dataset"><font face="Roboto">here</font></a>.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import log
import random

<font face="Times New Roman" size=4><div dir=ltr>
## Classification Decision Tree Class (60 points)
In this section, you only need to fill TODO parts. You can find the logic and formula in both course slides and the web, but fill it out on your own. 

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        if self.value is not None:
            return True
        return False

In [51]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def is_splitting_finished(self, depth, num_class_labels, num_samples):
        return depth >= self.max_depth or num_class_labels == 1 or num_samples < self.min_samples_split 

    def split(self, X, y, feature, threshold):
        left_indexes = X[feature] <= threshold
        right_indexes = -left_indexes
        X_left = X[left_indexes]
        y_left = y[left_indexes]
        X_right = X[right_indexes]
        y_right = y[right_indexes]

        return X_left, X_right, y_left, y_right

    def entropy(self, y):
        if len(y) ==0:
            return 0
        proportions = np.bincount(y.to_numpy().flatten()) / len(y)
        result = 0
        sub_sum = [np.log2(proportion) * proportion for proportion in proportions if proportion > 0]
        return -np.sum(sub_sum)


    def information_gain(self, X, y, feature, threshold):
        X_left, X_right, y_left, y_right = self.split(X, y, feature, threshold)
        H_y = self.entropy(y)
        p = len(X_left) / len(X)
        child_loss = p * self.entropy(y_left) + (1-p) * self.entropy(y_right)
        ig = H_y - child_loss
        return ig

    def best_split(self, X, y):
        final_score = float('-inf')
        final_feature = None
        final_thresh = None
        for feat in list(X.columns.values):
            thresholds = np.unique(X[feat])
            for threshold in thresholds:
                information_gain = self.information_gain(X, y, feat, threshold)
                if information_gain > final_score:
                    final_score = information_gain
                    final_feature = feat
                    final_thresh = threshold

        return final_feature, final_thresh

    def build_tree(self, X, y, depth=0):
        if self.is_splitting_finished(depth, len(X.columns), len(X)):
            return None

        best_feature, best_threshold = self.best_split(X, y)
        X_left, X_right, y_left, y_right = self.split(X, y, best_feature, best_threshold)

        left_node = self.build_tree(X_left, y_left, depth=depth + 1)
        right_node = self.build_tree(X_right, y_right, depth=depth + 1)

        value = None
        if left_node is None or right_node is None:
            true_value = len(y[y['diagnosis'] == 1])
            false_value = len(y[y['diagnosis'] == 0])
            if true_value >= false_value:
                value = 1
            else:
                value = 0

        return Node(feature=best_feature, threshold=best_threshold, left=left_node, right=right_node, value=value)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict(self, X):
        tree = self.root
        predicted_value = []
        for index in list(X.index):
            data = X.loc[index]
            current_tree = tree
            for depth in range(self.max_depth):
                if Node.is_leaf(current_tree):
                    predicted_value.append(current_tree.value)
                    break
                feature = current_tree.feature
                threshold = current_tree.threshold
                if data[feature] <= threshold:
                    current_tree = current_tree.left
                if data[feature] > threshold:
                    current_tree = current_tree.right

        return predicted_value

<font face="Times New Roman" size=4><div dir=ltr>
## Data Prepration (20 points)
In this section, you must perform a good EDA for data. Then split it into train and validation data. We will then use the validation data to find the best model hyperparameters.  

In [14]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv")

In [15]:
# TODO

In [16]:
X = breast_cancer_pdf.drop(['diagnosis'], axis=1)
y = breast_cancer_pdf[['diagnosis']]

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.70, random_state=42)

In [17]:
breast_cancer_pdf

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


<font face="Times New Roman" size=4><div dir=ltr>
## Training And Tuning Hyperparameters (20 points)
In this section, you only need to find the best hyperparameters for your model. You can test different values and permutations of hyperparameters by adding them to the lists below. Your model must have at least accuracy=0.85 on validation data.

In [53]:
max_depths = [i for i in range(1, 6)] #max value could be num of features
min_samples_splits = [i for i in range(2, 20, 3)]

In [54]:
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0
best_model = None
for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        clf = DecisionTree(max_depth, min_samples_split)
        clf.fit(x_train, y_train)
        y_val_pred = clf.predict(x_val)
        y_train_pred = clf.predict(x_train)
        accuracy = accuracy_score(y_val_pred, y_val)
        train_accuracy = accuracy_score(y_train_pred, y_train)
        print(f"accuracy of training set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={train_accuracy}")
        print(f"accuracy of validation set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={accuracy}")
        print("------------------------------------------------")
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_model = clf

accuracy of training set for [min_samples_splits=2-max_depths=1] =0.5764705882352941
accuracy of validation set for [min_samples_splits=2-max_depths=1] =0.6491228070175439
------------------------------------------------
accuracy of training set for [min_samples_splits=5-max_depths=1] =0.5764705882352941
accuracy of validation set for [min_samples_splits=5-max_depths=1] =0.6491228070175439
------------------------------------------------
accuracy of training set for [min_samples_splits=8-max_depths=1] =0.5764705882352941
accuracy of validation set for [min_samples_splits=8-max_depths=1] =0.6491228070175439
------------------------------------------------
accuracy of training set for [min_samples_splits=11-max_depths=1] =0.5764705882352941
accuracy of validation set for [min_samples_splits=11-max_depths=1] =0.6491228070175439
------------------------------------------------
accuracy of training set for [min_samples_splits=14-max_depths=1] =0.5764705882352941
accuracy of validation set f

In [55]:
print(f"Best result max depth: {best_max_depth}")
print(f"Best min samples split: {best_min_samples_split}")
print(f"Best accuracy: {best_accuracy}")

Best result max depth: 5
Best min samples split: 5
Best accuracy: 0.9047619047619048
